In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/Spring_2023_Project/COMS_572_Source_Code/CIFAR10"

/content/drive/MyDrive/Spring_2023_Project/COMS_572_Source_Code/CIFAR10


In [3]:
## load package
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# load build-in dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [6]:
## reshape and normalize data
x_train = x_train.reshape(50000, 32, 32, 3).astype("float32") / 255
x_test = x_test.reshape(10000, 32, 32, 3).astype("float32") / 255
y_train = y_train.reshape(50000,)
y_test = y_test.reshape(10000,)


In [7]:
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)

In [8]:
train_arr_75 = [tf.image.resize(img, (96, 96)) for img in x_train]
train_arr_75 = np.array(train_arr_75)
train_arr_75.shape

(50000, 96, 96, 3)

In [9]:
test_arr_75 = [tf.image.resize(img, (96, 96)) for img in x_test]
test_arr_75 = np.array(test_arr_75)
test_arr_75.shape

(10000, 96, 96, 3)

In [10]:
model = tf.keras.models.load_model('epoch10_stage_3_CIFAR10_VGG_build_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 3, 3, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                        

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 3, 3, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                        

In [12]:
model.layers[0].trainable=True
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 3, 3, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                        

In [13]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=opt,
      metrics=[tf.keras.metrics.CategoricalAccuracy()]
  )

In [14]:
obtained_learning_rate = np.loadtxt('epoch15_stage_2_lr.txt')

In [15]:
def scheduler(epoch):
  return obtained_learning_rate[epoch]
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [16]:
## Training the model Stage 3
with tf.device('/device:GPU:0'):
  final_history = model.fit(train_arr_75,y_train_one_hot, epochs=15, validation_data=(test_arr_75, y_test_one_hot), verbose=1, callbacks=[callback])

Epoch 1/2
1563/1563 [==============================] - 196s 113ms/step - loss: 0.3502 - categorical_accuracy: 0.8811 - val_loss: 0.4442 - val_categorical_accuracy: 0.8438 - lr: 1.0000e-04
Epoch 2/2
1563/1563 [==============================] - 180s 115ms/step - loss: 0.2317 - categorical_accuracy: 0.9207 - val_loss: 0.3809 - val_categorical_accuracy: 0.8767 - lr: 1.0000e-04


In [17]:
# evaluate the model
result = model.evaluate(test_arr_75, y_test_one_hot, verbose=1)

313/313 [==============================] - 10s 33ms/step - loss: 0.3809 - categorical_accuracy: 0.8767


In [18]:
result[1]*100

87.66999840736389

In [19]:
model.save('epoch10_15_final_CIFAR10_VGG_build_model.h5')